<a href="https://colab.research.google.com/github/Urdzik/Colab/blob/main/MDDL_Urdzik_CS26_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello World 2")

Hello World
